### 编辑距离的计算
编辑距离可以用来计算两个字符串的相似度，它的应用场景很多，其中之一是拼写纠正（spell correction）。 编辑距离的定义是给定两个字符串str1和str2, 我们要计算通过最少多少代价cost可以把str1转换成str2. 

举个例子：

输入:   str1 = "geek", str2 = "gesek"
输出:  1
插入 's'即可以把str1转换成str2

输入:   str1 = "cat", str2 = "cut"
输出:  1
用u去替换a即可以得到str2

输入:   str1 = "sunday", str2 = "saturday"
输出:  3

我们假定有三个不同的操作： 1. 插入新的字符   2. 替换字符   3. 删除一个字符。 每一个操作的代价为1. 

In [1]:
# 基于动态规划的方法
def get_mindistance(word1,word2):
    m = len(word1)
    n = len(word2)
    dp = [[0 for i in range(n+1)] for i in range(m+1)]
    
    # 初始化空字符串
    for i in range(1,m+1):
        dp[i][0] = i
    
    for i in range(1,n+1):
        dp[0][i] = i
        
    # 动态规划
    for i in range(1,m+1):
        for j in range(1,n+1):
            # 增加操作，str1a变为str2，然后再加上b，得到str2b
            insert_op = dp[i][j-1] + 1
            # 删除操作 ，str1a删除a后，再由str1变为str2b
            delete_op = dp[i-1][j] + 1
            # 替换操作 str1替换成str2
            replace_op = dp[i-1][j-1]
            # 添加判断a和b是否相等
            if word1[i-1] != word2[j-1]:
                replace_op += 1
            dp[i][j] = min(insert_op,delete_op,replace_op)
    return dp[m][n]

In [2]:
min_distance = get_mindistance('zhanghua','zhanghau')
print(min_distance)

2


### 生成指定编辑距离的单词
给定一个单词，我们也可以生成编辑距离为K的单词列表。 比如给定 str="apple"，K=1, 可以生成“appl”, "appla", "pple"...等
下面看怎么生成这些单词。 还是用英文的例子来说明。 仍然假设有三种操作 - 插入，删除，替换

In [3]:
# 老师给的例子
def generate_edit_one(str):
    """
    给定一个字符串，生成编辑距离为1的字符串列表。
    """
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(str[:i], str[i:])for i in range(len(str)+1)]
    inserts = [L + c + R for L, R in splits for c in letters]
    deletes = [L + R[1:] for L, R in splits if R]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    
    #return set(splits)
    return list(set(inserts + deletes + replaces))

print (len(generate_edit_one("apple")))

281


In [4]:
generate_edit_one("apple")[:5]

['sapple', 'apkple', 'aptple', 'apuple', 'applo']

In [5]:
# 自己写的生成编辑距离为1的单词候选集合
def get_edit_one_set(word):
    '''
    首先英文字母一共只有26个
    编辑距离无非是由三种操}作
    首先需要确定能够操作的位置
    首先假设word = app
    那么可以替换的位置有4个
    可以将word进行切割，切割后的集合为{('','app'),('a','pp'),('ap','p'),('app','')}
    1.insert操作，insert的操作可以在四个位置，例如添加c,'' + c + 'app',...
    2.delete操作，delete的操作可以在三个位置，最后一个split无法删除
    3.update操作，update的操作可以在三个位置，最后一个split无法替换
    '''
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i],word[i:]) for i in range(len(word) + 1)]
    
    insert_list = []
    delete_list = []
    update_list = []
    
    # 1.insert
    for (l,r) in splits:
        for c in letters:
            insert_list.append(l + c + r)
            
    # 2.delete
    for (l,r) in splits:
        if r:
            delete_list.append(l + r[1:])
    
    # 3.update
    for (l,r) in splits:
        if r:
            for c in letters:
                update_list.append(l + c + r[1:])
                
    return set(insert_list + delete_list + update_list)

In [6]:
one_edit_set = get_edit_one_set('apple')
print(len(one_edit_set))

281


In [7]:
# 老师给的例子,没考虑重复问题
def generate_edit_two(str):
    """
    给定一个字符串，生成编辑距离不大于2的字符串
    """
    return [e2 for e1 in generate_edit_one(str) for e2 in generate_edit_one(e1)]

print (len(generate_edit_two("apple")))
    

86524


In [8]:
# 自己写的生成编辑距离为2的字符串
def get_edit_two_set(word):
    '''
    先生成一个的，然后在候选集合继续生成编辑距离为1的集合
    '''
    result_list = []
    one_edit_set = get_edit_one_set(word)
    result_list.extend(list(one_edit_set))
    for w_o in one_edit_set:
        two_edit_set = get_edit_one_set(w_o)
        result_list.extend(list(two_edit_set))
        
    return set(result_list)

In [14]:
two_edit_set = get_edit_two_set('apple')
print(len(two_edit_set))

35334


### 基于结巴（jieba）的分词。 Jieba是最常用的中文分词工具~ 

In [9]:
# encoding=utf-8
import jieba

# 基于jieba的分词
seg_list = jieba.cut("贪心学院专注于人工智能教育", cut_all=False)
print("Default Mode: " + "/ ".join(seg_list))  

jieba.add_word("贪心学院")
seg_list = jieba.cut("贪心学院专注于人工智能教育", cut_all=False)
print("Default Mode: " + "/ ".join(seg_list)) 

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/l7/rbtg9dt10vd1vkf18t5zkpd00000gn/T/jieba.cache
Loading model cost 0.647 seconds.
Prefix dict has been built succesfully.


Default Mode: 贪心/ 学院/ 专注/ 于/ 人工智能/ 教育
Default Mode: 贪心学院/ 专注/ 于/ 人工智能/ 教育


### 判断一句话是否能够切分（被字典）

In [10]:
# 老师给的例子
dic = set(["贪心科技", "人工智能", "教育", "在线", "专注于"])
def word_break(str):
    could_break = [False] * (len(str) + 1)

    could_break[0] = True

    for i in range(1, len(could_break)):
        for j in range(0, i):
            if str[j:i] in dic and could_break[j] == True:
                could_break[i] = True

    return could_break[len(str)] == True

In [11]:
assert word_break("贪心科技在线教育")==True
assert word_break("在线教育是")==False
assert word_break("")==True
assert word_break("在线教育人工智能")==True

### 思考题：给定一个词典和一个字符串，能不能返回所有有效的分割？ （valid segmentation) 
比如给定词典：dic = set(["贪心科技", "人工智能", "教育", "在线", "专注于"， “贪心”])
和一个字符串 = “贪心科技专注于人工智能”

输出为： 
“贪心” “科技” “专注于” “人工智能”
"贪心科技" “专注于” “人工智能”

In [21]:
def all_possible_segmentations(str,vocab):
    '''
    思路：返回所有的可能集合
    首先对每个位置都建立集合list(list),外面的list代表有多少个分词方法，里面的list是具体的分词结构
    对于 贪心科技，该result 是[['贪心','科技'],['贪心科技']]
    '''
    
    '''
    debug之后才知道几个问题
    1.列表一定要复制
    2.list.copy()返回一个新列表
    '''
    
    # 子字符串的长度从1到len(str)
    # 动态维护
    break_list = [[] for i in range(len(str) + 1)]
    break_list[0].append([''])

    # 子字符串的长度从1到len(str)
    for i in range(1, len(str) + 1):
        # 子词开始的位置j,结束的位置i
        for j in range(0, i):
            if str[j:i] in vocab and len(break_list[j]) != 0:
                #print('j:%d  i:%d' % (j, i), break_list[j])
                for temp in break_list[j]:
                    a = temp.copy()
                    a.append(str[j:i])
                    break_list[i].append(a)
    
    result_list = []
    for temp in break_list[-1]:
        result_list.append(temp[1:])
    return result_list
 

vocab = set(["贪心科技", "人工智能", "教育", "在线", "专注于",'贪心','科技']) 
word = "贪心科技专注于人工智能"
print(all_possible_segmentations(word,vocab))

[['贪心科技', '专注于', '人工智能'], ['贪心', '科技', '专注于', '人工智能']]


### 停用词过滤
出现频率特别高的和频率特别低的词对于文本分析帮助不大，一般在预处理阶段会过滤掉。 
在英文里，经典的停用词为 “The”, "an".... 

In [14]:
# 方法1： 自己建立一个停用词词典
stop_words = ["the", "an", "is", "there"]
# 在使用时： 假设 word_list包含了文本里的单词
word_list = ["we", "are", "the", "students"]
filtered_words = [word for word in word_list if word not in stop_words]
print (filtered_words)

# 方法2：直接利用别人已经构建好的停用词库
#from nltk.corpus import stopwords
#cachedStopWords = stopwords.words("english")


['we', 'are', 'students']


In [15]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

test_strs = ['caresses', 'flies', 'dies', 'mules', 'denied',
         'died', 'agreed', 'owned', 'humbled', 'sized',
         'meeting', 'stating', 'siezing', 'itemization',
         'sensational', 'traditional', 'reference', 'colonizer',
         'plotted']

singles = [stemmer.stem(word) for word in test_strs]
print(' '.join(singles))  # doctest: +NORMALIZE_WHITESPACE

caress fli die mule deni die agre own humbl size meet state siez item sensat tradit refer colon plot


### 词袋向量： 把文本转换成向量 。 只有向量才能作为模型的输入。 

In [16]:
# 方法1： 词袋模型（按照词语出现的个数）
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
     'He is going from Beijing to Shanghai.',
     'He denied my request, but he actually lied.',
     'Mike lost the phone, and phone was in the car.',
]
X = vectorizer.fit_transform(corpus)

In [17]:
print (X.toarray())
print (vectorizer.get_feature_names())

[[0 0 1 0 0 0 1 1 1 0 1 0 0 0 0 0 0 1 0 1 0]
 [1 0 0 1 0 1 0 0 2 0 0 1 0 0 1 0 1 0 0 0 0]
 [0 1 0 0 1 0 0 0 0 1 0 0 1 1 0 2 0 0 2 0 1]]
['actually', 'and', 'beijing', 'but', 'car', 'denied', 'from', 'going', 'he', 'in', 'is', 'lied', 'lost', 'mike', 'my', 'phone', 'request', 'shanghai', 'the', 'to', 'was']


In [18]:
# 方法2：词袋模型（tf-idf方法）
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(smooth_idf=False)
X = vectorizer.fit_transform(corpus)

In [19]:
print (X.toarray())
print (vectorizer.get_feature_names())

[[0.         0.         0.39379499 0.         0.         0.
  0.39379499 0.39379499 0.26372909 0.         0.39379499 0.
  0.         0.         0.         0.         0.         0.39379499
  0.         0.39379499 0.        ]
 [0.35819397 0.         0.         0.35819397 0.         0.35819397
  0.         0.         0.47977335 0.         0.         0.35819397
  0.         0.         0.35819397 0.         0.35819397 0.
  0.         0.         0.        ]
 [0.         0.26726124 0.         0.         0.26726124 0.
  0.         0.         0.         0.26726124 0.         0.
  0.26726124 0.26726124 0.         0.53452248 0.         0.
  0.53452248 0.         0.26726124]]
['actually', 'and', 'beijing', 'but', 'car', 'denied', 'from', 'going', 'he', 'in', 'is', 'lied', 'lost', 'mike', 'my', 'phone', 'request', 'shanghai', 'the', 'to', 'was']
